In [ ]:
import pandas as pd

df = pd.read_csv("combined_redditAPI.csv")
print(df.head())

  subreddit       created                                              title  \
0    stocks  1740823242.0  Rate My Portfolio - r/Stocks Quarterly Thread ...   
1    stocks  1744623032.0    r/Stocks Daily Discussion Monday - Apr 14, 2025   
2    stocks  1744638624.0  If the economy is crashing, why is no one acti...   
3    stocks  1744647937.0  Genuine question: How has the US market not co...   
4    stocks  1744656215.0  Trump considers pausing his auto tariffs as th...   

                                            selftext  upvote_ratio   ups  \
0  Please use this thread to discuss your portfol...          0.95    87   
1  These daily discussions run from Monday to Fri...          1.00    20   
2  My apologies if this is not a smart question, ...          0.89  4599   
3  I just don't get it. With tariffs and policies...          0.88  1647   
4  [https://finance.yahoo.com/news/trump-consider...          0.97   824   

   downs  score  
0      0     87  
1      0     20  
2      0

In [ ]:
df

,subreddit,created,title,selftext,upvote_ratio,ups,downs,score
0,stocks,1740823242.0,Rate My Portfolio - r/Stocks Quarterly Thread ...,Please use this thread to discuss your portfol...,0.95,87,0,87
1,stocks,1744623032.0,"r/Stocks Daily Discussion Monday - Apr 14, 2025",These daily discussions run from Monday to Fri...,1.00,20,0,20
2,stocks,1744638624.0,"If the economy is crashing, why is no one acti...","My apologies if this is not a smart question, ...",0.89,4599,0,4599
3,stocks,1744647937.0,Genuine question: How has the US market not co...,I just don't get it. With tariffs and policies...,0.88,1647,0,1647
4,stocks,1744656215.0,Trump considers pausing his auto tariffs as th...,[https://finance.yahoo.com/news/trump-consider...,0.97,824,0,824
...,...,...,...,...,...,...,...,...
909,wallstreetbets,1744344882.0,Tesla suspends taking new orders for Model S a...,NaN,0.98,1804,0,1804
910,wallstreetbets,1744349875.0,China selling off US treasuries = calls on tre...,"So, its evident China is stabilizing their fin...",0.91,1065,0,1065
911,wallstreetbets,1744365441.0,"Daily Discussion Thread for April 11, 2025",This post contains content not supported on ol...,0.93,422,0,422
912,wallstreetbets,1744399571.0,Moon or Wendi’s?,NaN,0.95,70,0,70


In [ ]:
def strip_minutes_seconds_keep_hour(df, column):
    df[column] = pd.to_datetime(df[column], unit='s', errors='coerce')
    df[column] = df[column].dt.floor('H')
    df[column] = df[column].dt.strftime('%Y-%m-%d %H:00')
    return df

df = strip_minutes_seconds_keep_hour(df, 'created')


/var/folders/sm/7sj4nhxs3pl0jc1lhw83y4yc0000gn/T/ipykernel_90451/1658137617.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[column] = pd.to_datetime(df[column], unit='s', errors='coerce')
/var/folders/sm/7sj4nhxs3pl0jc1lhw83y4yc0000gn/T/ipykernel_90451/1658137617.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df[column] = df[column].dt.floor('H')


In [ ]:
import requests
import pandas as pd
import json
import re
from tqdm import tqdm

df["Is_controversial"] = 0

system_prompt = (
    "You are a helpful assistant that classifies a single tweet into 'controversial' (1) "
    "or 'non-controversial' (0). Return ONLY the digit 1 or 0."
)

definition = (
    "Definition of 'controversial': Controversial content refers to any statement, idea, or expression that is likely to "
    "provoke strong disagreement, debate, or emotional reactions. It often involves sensitive, polarizing, or divisive topics, "
    "and typically reflects opinionated, critical, or challenging viewpoints.\n\n"
)

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Classifying"):
    news = row["selftext"]
    user_prompt = (
        definition
        + f"News: \"{news}\"\n\n"
        "Please classify this tweet as controversial (1) or non-controversial (0). "
        "Return ONLY the digit 1 or 0."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ]

    val = 0

    try:
        response = requests.post(
            "http://localhost:11434/api/chat",
            json={
                "model": "llama3",
                "messages": messages,
                "options": {"temperature": 0}
            },
            stream=True
        )

        full_response = ""
        for line in response.iter_lines():
            if not line:
                continue
            try:
                part = json.loads(line.decode("utf-8"))
                content = part.get("message", {}).get("content", "")
                full_response += content
            except json.JSONDecodeError:
                continue

        m = re.search(r"\b(1|0)\b", full_response.strip())
        if m:
            val = int(m.group(1))

    except Exception as e:
        print(f"[Warning] failed at index {idx}: {e}")

    df.at[idx, "Is_controversial"] = val

print(df.head())

Classifying: 100%|████████████████████████████| 914/914 [07:10<00:00,  2.12it/s]

  subreddit           created  \
0    stocks  2025-03-01 10:00   
1    stocks  2025-04-14 09:00   
2    stocks  2025-04-14 13:00   
3    stocks  2025-04-14 16:00   
4    stocks  2025-04-14 18:00   

                                               title  \
0  Rate My Portfolio - r/Stocks Quarterly Thread ...   
1    r/Stocks Daily Discussion Monday - Apr 14, 2025   
2  If the economy is crashing, why is no one acti...   
3  Genuine question: How has the US market not co...   
4  Trump considers pausing his auto tariffs as th...   

                                            selftext  upvote_ratio   ups  \
0  Please use this thread to discuss your portfol...          0.95    87   
1  These daily discussions run from Monday to Fri...          1.00    20   
2  My apologies if this is not a smart question, ...          0.89  4599   
3  I just don't get it. With tariffs and policies...          0.88  1647   
4  [https://finance.yahoo.com/news/trump-consider...          0.97   824   

   down

In [ ]:
df.to_csv("classified_reddit.csv", index=False)